# Reproduction of Tangri et al.'s Kidney Failure Risk Equation (KFRE)

## Import Libraries

In [1]:
########################### Import Requisite Libraries #########################
import pandas as pd
import numpy as np
import os
import sys
import pprint

################################################################################
# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))
from python_scripts.functions import *
from python_scripts.kfre import *

## Read in the Data

In [2]:
base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder
data_path = os.path.join(os.pardir, "data")

image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

# Read the CSV file into a DataFrame
df = pd.read_parquet(os.path.join(data_path, "df.parquet"))
df_eda = pd.read_parquet(os.path.join(data_path, "df_eda.parquet"))

Directory exists: ..\data
Directory exists: ..\images\png_images
Directory exists: ..\images\svg_images


In [3]:
# join back sex to original dataframe for eda dataframe so it can be used
df = df.join(df_eda["SEX"], on="Patient_ID", how="inner")

In [4]:
df_kfre = df.copy(deep=True)

In [5]:
df_kfre.columns

Index(['Age', 'Diabetes (1=yes; 0=no)', 'Hypertension (1=yes; 0=no)',
       'Renal disease (DM=1, HTN=2, GN=3, ADPKD=4, Other=5)', 'eGFR-EPI',
       'uPCR', 'Calcium (mmol/L)', 'Phosphate (mmol/L)',
       'Bicarbonate (mmol/L)', 'Albumin (g/l)', 'Follow-up YEARS', 'RIP',
       'ESRD', 'SEX_Female', 'SEX_Male',
       'ETHNICITY_AFRICAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_ANY OTHER ASIAN BACKGROUND',
       'ETHNICITY_ANY OTHER BLACK BACKGROUND',
       'ETHNICITY_ANY OTHER ETHNIC GROUP',
       'ETHNICITY_ANY OTHER MIXED BACKGROUND',
       'ETHNICITY_ANY OTHER WHITE BACKGROUND',
       'ETHNICITY_BANGLADESHI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_CARIBBEAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_CHINESE (OTHER ETHNIC GROUPS)',
       'ETHNICITY_INDIAN (ASIAN OR ASIAN BRITISH)', 'ETHNICITY_NOT STATED',
       'ETHNICITY_PAKISTANI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_WHITE AND BLACK AFRICAN (MIXED)', 'ETHNICITY_WHITE BRITISH',
       'ETHNICITY_WHITE IRIS

In [6]:
[col for col in df.columns if "Bicarb" in col]

['Bicarbonate (mmol/L)']

In [7]:
df["Bicarbonate (mmol/L)"].median()

21.8

In [8]:
# subset the dataframe to include only relevant variables for the KFRE equation
# Age, eGFR, log-normalized uACR, Diabetes, Hypertension
df_kfre = df_kfre[
    [
        "Age",
        "SEX",
        "eGFR-EPI",
        "uACR",
        "Diabetes (1=yes; 0=no)",
        "Hypertension (1=yes; 0=no)",
        "Albumin (g/dL)",
        "Calcium (mg/dL)",
        "Phosphate (mg/dL)",
        "Bicarbonate (mmol/L)",
        "ESRD",
    ]
]

In [9]:
columns = {
    "age": "Age",
    "sex": "SEX",
    "eGFR": "eGFR-EPI",
    "uACR": "uACR",
    "region": "Region",
    "dm": "Diabetes (1=yes; 0=no)",
    "htn": "Hypertension (1=yes; 0=no)",
    "albumin": "Albumin (g/dL)",
    "phosphorous": "Phosphate (mg/dL)",
    "bicarbonate": "Bicarbonate (mmol/L)",
    "calcium": "Calcium (mg/dL)",
}

predictor = RiskPredictor(data=df_kfre, columns=columns)

# Predict risk for different combinations of years and variables
pred_4var_2year = predictor.predict_kfre(years=2, is_north_american=False)
pred_4var_5year = predictor.predict_kfre(years=5, is_north_american=False)
pred_6var_2year = predictor.predict_kfre(
    years=2, is_north_american=False, use_extra_vars=True, num_vars=6
)
pred_6var_5year = predictor.predict_kfre(
    years=5, is_north_american=False, use_extra_vars=True, num_vars=6
)

pred_8var_2year = predictor.predict_kfre(
    years=2, is_north_american=False, use_extra_vars=True, num_vars=8
)
pred_8var_5year = predictor.predict_kfre(
    years=5, is_north_american=False, use_extra_vars=True, num_vars=8
)

# inner join the different combinations of years and variables to
# kfre_inclusion dataframe
df_kfre = df_kfre.assign(
    pred_4var_2year=pred_4var_2year,
    pred_4var_5year=pred_4var_5year,
    pred_6var_2year=pred_6var_2year,
    pred_6var_5year=pred_6var_5year,
    pred_8var_2year=pred_8var_2year,
    pred_8var_5year=pred_8var_5year,
)

In [10]:
df_kfre.head()

,Age,SEX,eGFR-EPI,uACR,Diabetes (1=yes; 0=no),Hypertension (1=yes; 0=no),Albumin (g/dL),Calcium (mg/dL),Phosphate (mg/dL),Bicarbonate (mmol/L),ESRD,pred_4var_2year,pred_4var_5year,pred_6var_2year,pred_6var_5year,pred_8var_2year,pred_8var_5year
Patient_ID,,,,,,,,,,,,,,,,,
568268916,87.24,Male,19.0,102.435547,1.0,1.0,3.7,11.12,2.728,27.2,0,0.067174,0.236054,0.064697,0.224182,0.029092,0.124365
659549633,56.88,Female,15.0,1762.001840,0.0,1.0,3.0,9.72,3.162,21.3,1,0.449505,0.900884,0.485949,0.919974,0.413625,0.909390
406748956,66.53,Female,17.0,659.122070,0.0,1.0,3.6,9.32,3.844,27.8,0,0.219609,0.617166,0.241500,0.649720,0.157297,0.536917
109228713,69.92,Male,12.0,1145.220631,0.0,1.0,3.9,9.16,5.580,20.7,1,0.482796,0.922154,0.520481,0.938537,0.599697,0.983729
220533110,81.14,Female,15.0,980.918742,1.0,1.0,4.3,9.80,4.309,26.2,0,0.235647,0.646744,0.227775,0.625051,0.147546,0.512321


In [11]:
df["Bicarbonate (mmol/L)"].median()

21.8

In [12]:
df_kfre["pred_4var_2year"].median()

0.23065730964986586

In [13]:
df_kfre["pred_8var_5year"].median()

0.6396378292796349